# Sales Outreach Agent

Sales outreach is one of the most practicle business use cases for LLM agents.
Companies need to reach out to potential clients in a variety of tones from formal to fun. In this example , well :

* Define multiple agents, each with a unique communication style 
* Prompt them to write cold emails using OpenAI's GPT models
* Stream the generated response in real time 
* (Optional) Send the email using SendGrid

In [8]:
from dotenv import load_dotenv
from agents import Agent , Runner, trace
from openai.types.responses import ResponseTextDeltaEvent


load_dotenv(override=True)



True

In [9]:
# Define agent persona with distinct tones

professional_prompt = (
    "You are a professional agent at TechLink,"
    "an online education company offering AI-Driven , carrer-focused learning programs. "
    "Your goal is to write serious, persuasive cold outreach emails targeted at professionals and educational institutions."
)


engaging_prompt = (
    "You are a witty, engaging sales agent at TechLink, "
    "an online education company offering AI-powered, upskilling programs for tech and business roles. "
    "Craft humorous, attention-grabbing cold emails that spark curiosity and encourage responses."
)

concise_prompt = (
    "You are a fast-moving, efficient sales agent at TechLink, "
    "an online education platform offering smart, AI-powered learning paths. "
    "Write short, compelling cold emails that quickly highlight value and drive action."
)

In [10]:
# create 3 disctinct agents

professional_agent = Agent(
    name="Professional Sales Agent",
    instructions=professional_prompt,
    model="gpt-4o-mini"
)

engaging_agent = Agent(
    name="Engaging Sales Agent",
    instructions=engaging_prompt,
    model="gpt-4o-mini"
)

concise_agent = Agent(
    name="Concise Sales Agent",
    instructions=concise_prompt,
    model="gpt-4o-mini"
)

In [11]:
# Stream and display email content from an agent
email_result = Runner.run_streamed(engaging_agent, input="Write a cold sales email")
async for event in email_result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: 🚀 Ready to Upgrade Your Skills? No Rocket Science Required!

Hi [Recipient's Name],

Ever tried to fix your Wi-Fi by yelling at it? Spoiler: It doesn’t work. But if you’re looking to enhance your skills instead—without the tech tantrums—I've got just the ticket!

At TechLink, we're serving up AI-powered upskilling programs that make learning feel more like a Netflix binge and less like a math exam. From leveling up your coding skills to mastering the mysteries of business strategy, we’ve got courses that even your pet goldfish would nod off during (no offense to goldfish, they’re wonderful creatures).

🧠 Imagine this: You, conquering workplace challenges with newfound prowess, while your colleagues are still trying to figure out Zoom filters. 

Curious to learn how we can help you level up? Hit reply, and let’s chat! I promise I'll keep the tech jargon to a minimum (unless you ask for it). 

Best,

[Your Name]  
TechLink | Level Up Your Skills  
[Your Contact Information]  

P

In [12]:
import asyncio

# The email topic or objective
email_topic = "Write a cold sales email"

# Run all sales agents concurrently to generate email variations
agent_outputs = await asyncio.gather(
    Runner.run(professional_agent,email_topic),
    Runner.run(engaging_agent,email_topic),
    Runner.run(concise_agent,email_topic),
)

# Extract the final text output from each agent
cold_emails = [response.final_output for response in agent_outputs]

# Display all generated emails
for idx, email in enumerate(cold_emails, start=1):
    print(f"\n--- Email Option {idx} ---\n{email}\n")


--- Email Option 1 ---
Subject: Transform Your Team’s Skillset with AI-Driven Learning

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent TechLink, a leading online education company specializing in AI-driven, career-focused learning programs.

In today’s fast-paced digital landscape, staying competitive requires not just adapting to changes but anticipating them. At TechLink, we understand the challenges organizations face in upskilling their workforce efficiently and effectively. Our tailored programs harness the power of artificial intelligence to provide personalized learning paths that match your team’s unique needs and career aspirations.

Imagine a training solution that not only equips your professionals with the latest industry skills but also engages them through interactive, real-world scenarios. From data analytics to machine learning and beyond, our courses are designed to empower your team to excel in their roles and co

In [13]:
# Create an evaluator agent to choose the most appealing email
selector_prompt = (
    "You are a prospective customer receiving cold outreach emails. "
    "Choose the email you're most likely to respond to, based solely on content. "
    "Don't explain your choice—just return the best email as-is."
)

email_selector_agent = Agent(
    name="Email Selector Agent",
    instructions=selector_prompt,
    model="gpt-4o-mini"
)

# Evaluate all email options and pick the most compelling one
with trace("Evaluate & select best cold email"):
    agent_outputs = await asyncio.gather(
        Runner.run(professional_agent,email_topic),
        Runner.run(engaging_agent,email_topic),
        Runner.run(concise_agent,email_topic),
    )

    # Extract email outputs
    email_variants = [res.final_output for res in agent_outputs]

    # Combine all into a single prompt for the selector agent
    evaluation_prompt = "Evaluate the following cold sales emails:\n\n" + "\n\n---\n\n".join(email_variants)

    # Run selector agent
    selected_response = await Runner.run(email_selector_agent, evaluation_prompt)
    
    print("\nBest Sales Email Selected:\n")
    print(selected_response.final_output)


Best Sales Email Selected:

Subject: Ready to Upgrade Your Brain? 💡

Hey [Recipient's Name],

Hope this email finds you well, and that your coffee is stronger than your Wi-Fi signal! ☕️💻

I’m reaching out from TechLink – the online learning platform that’s like a gym membership for your brain, but without the sweat and awkward locker room moments! 🎓

In today’s fast-paced tech world, keeping your skills sharp is a bit like trying to outrun a cheetah on roller skates. 🐆🛼 But don’t worry—I’ve got just the solution!

Imagine taking AI-powered courses that fit your busy schedule, turning you into the tech-savvy wizard your team secretly believes you are. ✨ Whether you want to master data analytics, command the realm of coding, or simply impress your cat with your newfound knowledge, we’ve got something for you! 

Curious? Let’s chat! I promise there will be no awkward small talk about the weather—unless you want to!

Looking forward to hearing from you (and maybe sharing a virtual cup of 

# Use Agents and Function as Tool

In [ ]:
import sendgrid
import os
from sendgrid.helpers.mail import Email, To, Content, Mail
from agents import function_tool, Agent, trace

# Define a tool function to send an email
@function_tool
def send_sales_email(email_body: str):
     """Send the provided email body to the sales prospect."""
     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
     sender = Email("anshulc55@gmail.com")
     recipient = To("anshulc55@icloud.com")
     content = Content("text/plain", email_body)
     email = Mail(sender, recipient, "Cold Outreach from TechLink", content).get()
     response = sg.client.mail.send.post(request_body=email)
     return {"status": "sent"}

In [ ]:
send_sales_email

# Convert Agents into Tools

In [ ]:
# Wrap agents as callable tools

tool_pro = professional_agent.as_tool(
    tool_name="professional_email_generator",
    tool_description="Generates a serious, professional cold email."
)

tool_fun = engaging_agent.as_tool(
    tool_name="witty_email_generator",
    tool_description="Creates a humorous and engaging sales email."
)

tool_brief = concise_agent.as_tool(
    tool_name="brief_email_generator",
    tool_description="Produces a short, direct sales email."
)

# Define the full toolset available to the manager agent
toolset = [tool_pro, tool_fun, tool_brief, send_sales_email]

# Define the Sales Manager Agent with tool-based instructions

In [ ]:
# Sales Manager only uses tools — no email writing on its own
manager_prompt = (
    "You are a sales manager at TechLink, responsible for outbound outreach. "
    "You are provided with several email-writing tools, each with a unique style. "
    "Your task is to try all three once, evaluate which email is the best for the user, "
    "and then use the 'send_sales_email' tool to send only the best email. "
    "Do NOT generate your own email — only use tools."
)

sales_manager_agent = Agent(
    name="Sales Manager",
    instructions=manager_prompt,
    tools=toolset,
    model="gpt-4o-mini"
)

# Use the Sales Manager to run the end-to-end orchestration
message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales Manager Workflow"):
    final_result = await Runner.run(sales_manager_agent, message)

# HandOff Workflows

## Hybrid use of tools and Handoffs
* Collaborating with tools (agent-as-a-tool) and delegating control entirely (handoff)



## Agent as a Tool vs Handoff

* Agent as Tool -
 1. Agent is used like a function or utility
 2. Control returns to the caller after use.
 3. One-off utility behaviour (e.g Subject writing , formatting)
 4. Used in tools=[...]

* Handoff -
 1. Agent is used like a delegate or collaborator
 2. Control passes to the handoff agent permanently
 3. Full workflows requiring automony (e.g email formatting + sending)
 4. Used in handoff = [....]

In [ ]:
from agents import Agent

# Subject Line Generator
subject_agent = Agent(
    name="SubjectLineWriter",
    instructions="Generate a subject line for a cold email that improves open rates.",
    model="gpt-4o-mini"
)

# HTML Converter Agent
html_agent = Agent(
    name="HTMLFormatter",
    instructions="Convert plain text emails into well-structured, visually clear HTML format.",
    model="gpt-4o-mini"
)

subject_tool = subject_agent.as_tool(
    tool_name="generate_subject", 
    tool_description="Generate a cold email subject line."
    )

html_tool = html_agent.as_tool(
    tool_name="convert_to_html", 
    tool_description="Convert text email to HTML."
    )

# Function Tool to Send Email via SendGrid
@function_tool
def send_email(subject: str, html_body: str):
     """Send an HTML email with subject and body to the recipient list."""
     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
     sender = Email("anshulc55@gmail.com")
     recipient = To("anshulc55@icloud.com")
     content = Content("text/html", html_body)
     email = Mail(sender, recipient, subject, content).get()
     response = sg.client.mail.send.post(request_body=email)
     return {"status": "sent"}

# Combine all tools for formatting and sending
email_formatter_tools = [subject_tool, html_tool, send_email]
email_formatter_tools

## Create a Specialized Email Manager Agent (Handoff Target)

In [ ]:
email_manager_prompt = (
        "You are responsible for taking a raw email body and sending it out. "
        "First, use the subject generator tool. Then, convert the email to HTML format. "
        "Finally, use the send_email tool to deliver the message."
    )

# HandOff Agent
email_manager_agent = Agent(
    name="EmailManager",
    instructions=email_manager_prompt,
    tools=email_formatter_tools,
    model="gpt-4o-mini",
    handoff_description="Format and send an email using subject generation and HTML conversion."
)


# These are your content generators
email_generator_tools = [tool_pro, tool_fun, tool_brief]

## Sales Manager Agent Using Both Tools & Handoff

In [ ]:
# The Sales Manager Agent coordinates everything
sales_manager_prompt = (
        "You are a sales manager tasked with sending the best possible cold email from your company TechLink. "
        "Use the available email-writing tools to generate several options. "
        "Choose the most persuasive email. Then, hand off to the EmailManager agent to format and send the email."
    )

sales_manager_agent = Agent(
    name="SalesManager",
    instructions=sales_manager_prompt,
    tools=email_generator_tools,
    handoffs=[email_manager_agent],
    model="gpt-4o-mini"
)

# Run the Full Workflow
prompt = "Send out a cold sales email addressed to Dear CEO from Anshul."
with trace("Automated Sales Email Flow"):
    result = await Runner.run(sales_manager_agent, prompt)